In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization
import matplotlib.pyplot as plt

### Step 1: Data loading and preprocessing

In [ ]:
data_dir = "path/to/your/dataset"

# Label is inferred by the folder (office or screenshot)
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels = 'inferred',
    label_mode = 'binary',
    image_size = (128, 128),
    batch_size = 32,
    shuffle = True
)

# Display the total number of images of the dataset
total_images = 0
for images, labels in dataset:
    total_images += images.shape[0]

print(f'Total number of images in the dataset: {total_images}')

In [ ]:
# Plot some images to visualize how they will be input into the model
class_names = dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
    for i in range(6):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label_index = int(labels[i])
        plt.title(class_names[label_index])
        plt.axis("off")
plt.show()

In [ ]:
# Normalization to [0, 1]
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255
    return image, label

dataset = dataset.map(normalize)

In [ ]:
# Split into training and validation datasets
dataset_size = len(list(dataset))
train_size = int(0.8 * dataset_size)

train_dataset = dataset.take(train_size)
valid_dataset = dataset.skip(train_size)

### Step 2: Define model architecture

In [ ]:
# Model will be a CNN with thre convolutional blocks and the head
modelCNN = keras.Sequential([
    # First block
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.MaxPool2D(),

    # Second block
    layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.MaxPool2D(),

    # Third block
    layers.Conv2D(128, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.Conv2D(128, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.MaxPool2D(),

    # Classifier head
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(units=50, activation='relu'),
    layers.Dense(units=2, activation='sigmoid')
])

In [ ]:
modelCNN.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

### Step 3: Train the model

In [ ]:
history = modelCNN.fit(train_dataset, validation_data=valid_dataset, epochs=10)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(False)
plt.show()

### Step 4: Evaluate and save

In [ ]:
loss, accuracy = modelCNN.evaluate(valid_dataset)
print(f'Validation accuracy: {accuracy:.2f}')

In [ ]:
modelCNN.save('my_cnn_model.keras')